In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

from IPython.display import clear_output
from datetime import datetime

In [2]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./1.page.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['Page'] = input_['Page'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        for b in range(666):
            try:
                import requests

                import sys
                sys.path.append('../../00.Tools')
                from crawler_configuration import get_proxy

                resp = requests.get(input_.loc[a, 'Page Url'],
                                    proxies=get_proxy(),
                                    timeout=(10, 10)).text

                # = = = = = = = = = = = = = = =

                import json

                json_ = json.loads(resp.split('$MC=(window.$MC||[]).concat(')[1].split(']}]})</script>')[0].strip() + ']}]}')

                # = = = = = = = = = = = = = = =

                list_item_number = [w[2]['model']['action']['params']['itemId'].strip() if type(w[2]) == dict and 'model' in w[2] and 'action' in w[2]['model'] and 'params' in w[2]['model']['action'] and 'itemId' in w[2]['model']['action']['params'] else '' for w in json_['o']['w']]
                while '' in list_item_number:
                    list_item_number.remove('')

                # = = = = = = = = = = = = = = =

                if len(list_item_number) == 0:
                    continue

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame({'Market': input_.loc[a, 'Market'],
                                        'Page': input_.loc[a, 'Page'],
                                        'Page Url': input_.loc[a, 'Page Url'],
                                        'No': [i+1 for i in range(len(list_item_number))],
                                        'Item Number': list_item_number})

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'

                # = = = = = = = = = = = = = = =

                break
            except:
                continue

        # = = = = = = = = = = = = = = =

        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Market': input_.loc[a, 'Market'],
                                     'Page': input_.loc[a, 'Page'],
                                     'Page Url': input_.loc[a, 'Page Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b+1:3d}] - {input_.loc[a, 'Page']}.{input_.loc[a, 'Page Url']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(12):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['Page', 'No'],
                                                ascending=[True, True],
                                                ignore_index=True)
    output_correct = output_correct.drop_duplicates(['Item Number'],
                                                    keep='first',
                                                    ignore_index=True)
    output_correct['No'] = [i+1 for i in range(len(output_correct))]
    output_correct.to_excel(f'''./2.menu-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['Page'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel(f'./menu_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：3

[状态：ok，尝试次数：  1] - 3.https://www.ebay.co.uk/sch/i.html?_dkr=1&iconV2Request=true&_blrs=recall_filtering&_ssn=spares-hut&store_cat=0&store_name=allautopartsonline&_oac=1&_nkw=mirror&orig_cvip=true&_sop=16&_ipg=240&_udlo=&_udhi=&_ul=GB&_stpos=BS81QU&_pgn=3
[剩余数量：0] - [当前时间：19:25:23]

[状态：ok，尝试次数：  1] - 1.https://www.ebay.co.uk/sch/i.html?_dkr=1&iconV2Request=true&_blrs=recall_filtering&_ssn=spares-hut&store_cat=0&store_name=allautopartsonline&_oac=1&_nkw=mirror&orig_cvip=true&_sop=16&_ipg=240&_udlo=&_udhi=&_ul=GB&_stpos=BS81QU&_pgn=1
[剩余数量：0] - [当前时间：19:25:26]

[状态：ok，尝试次数：  1] - 2.https://www.ebay.co.uk/sch/i.html?_dkr=1&iconV2Request=true&_blrs=recall_filtering&_ssn=spares-hut&store_cat=0&store_name=allautopartsonline&_oac=1&_nkw=mirror&orig_cvip=true&_sop=16&_ipg=240&_udlo=&_udhi=&_ul=GB&_stpos=BS81QU&_pgn=2
[剩余数量：0] - [当前时间：19:25:27]

输出ing...

Done ~
